# Make response data (DR_Response_demo.csv)

In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import h5py
from dots_reversal import DotsReversal
from dots_reversal_util import lam2W_H, geometric_mean

In [2]:
# Parameters
labels = ["Low", "High"]
difficulties = ["easy", "difficult"]
refresh_rate = 60
k = 0.5
L_range = 10
dL = 0.4

In [3]:
# Get trials
trials_param = pd.read_csv("../trial_data/trials_param_modulate_hazard_rate.csv")
trials = pd.read_csv("../trial_data/trials_modulate_hazard_rate.csv")
trial_mask = ~trials.reversal0.isna() # Mask for trials with at least one reversal
trial_mask_low = trial_mask & (trials_param.hazard_rate_state == "easy")
trial_mask_high = trial_mask & (trials_param.hazard_rate_state == "difficult")
trials_low = trials[trial_mask_low].reset_index(drop=True)
trials_high = trials[trial_mask_high].reset_index(drop=True)
print("Number of trials:", trials_low.shape[0], trials_high.shape[0])
lam_geomean_low = geometric_mean(torch.from_numpy(trials_low.hazard_rate.values))
lam_geomean_high = geometric_mean(torch.from_numpy(trials_high.hazard_rate.values))
print("Geometric Average λ:", lam_geomean_low, lam_geomean_high)

# Set parameters and optimizer
W_H_low = lam2W_H(lam_geomean_low, refresh_rate)
W_H_high = lam2W_H(lam_geomean_high, refresh_rate)
W_k = torch.log(torch.tensor(k))
model_low = DotsReversal(W_H_low, W_k, trials_low, refresh_rate, L_range, dL, requires_grad = False)
model_high = DotsReversal(W_H_high, W_k, trials_high, refresh_rate, L_range, dL, requires_grad = False)
responses = torch.zeros(trials.shape[0])
loss = 0
for i_env in range(len(labels)):
    label = labels[i_env]
    difficulty = difficulties[i_env]
    model = [model_low, model_high][i_env]
    pL_right_trials = model.predict()
    trial_env_mask = trial_mask & (trials_param.hazard_rate_state == difficulty)
    responses[trial_env_mask] = (0.5 <= pL_right_trials).type(torch.float)
    loss = loss + model.loss(pL_right_trials, responses[trial_env_mask]).mean()

responses_demo = responses.detach().numpy().astype(str)
responses_demo[responses_demo == "0.0"] = "f"
responses_demo[responses_demo == "1.0"] = "j"
assert np.all(np.isin(responses_demo, ["f", "j"]))
subject_ids = np.zeros(trials.shape[0]).astype(int)
trial_ids = np.arange(trials.shape[0]) + 1
df = pd.DataFrame({"subject_id": subject_ids, "trial_id": trial_ids, "response": responses_demo})
df.to_csv("DR_Response_demo.csv", index=False)

Number of trials: 81 128
Geometric Average λ: tensor(0.1992) tensor(3.0053)
